In [223]:
# Data
import numpy as np
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import joblib

import warnings
warnings.filterwarnings('ignore')


In [224]:
df = pd.read_csv("data/test.csv")
df.sample(5)

,App Name,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genre,Last Updated,Current Version,Android Version
1802,Beauty Idol: Fashion Queen,GAME,128367,54M,"1,000,000+",Free,0,Everyone,Arcade,"October 27, 2017",2.1.0,4.0.3 and up
987,PUBG MOBILE,GAME,3716278,36M,"50,000,000+",Free,0,Teen,Action,"July 24, 2018",0.7.0,4.3 and up
429,Five nights at Minecraft,FAMILY,9335,12M,"1,000,000+",Free,0,Everyone,Entertainment,"November 26, 2017",1.2,4.0.3 and up
130,Granny,GAME,1137271,59M,"50,000,000+",Free,0,Teen,Arcade,"May 29, 2018",1.4.0.1,4.0 and up
1544,Death Dragon Knights RPG,FAMILY,11179,19M,"1,000,000+",Free,0,Teen,Role Playing,"September 16, 2016",2.7,2.3 and up


Reviews

Size

In [225]:
df['Installs'] = df['Installs'].str.replace('+', '', regex=False)
df['Installs'] = df['Installs'].str.replace(',', '', regex=False)


In [226]:
df['Price'] = df['Price'].str.replace('$', '', regex=False)


In [227]:
df['Installs'] = pd.to_numeric(df['Installs'])


In [228]:
df['Price'] = pd.to_numeric(df['Price'])


last updated

In [229]:
# Convert to datetime format
df['Last Updated'] = pd.to_datetime(df['Last Updated'])

# Feature 1: Days since last update (relative to the most recent date)
most_recent_date = df['Last Updated'].max()
df['Days_Since_Update'] = (most_recent_date - df['Last Updated']).dt.days
most_recent_date

Timestamp('2018-08-07 00:00:00')

In [230]:
df = df.drop('Last Updated', axis=1)


In [231]:
df['Is_Varies_Android'] = df['Android Version'] == 'Varies with device'
df['Is_Varies_Current'] = df['Current Version'] == 'Varies with device'


they are significant

In [232]:
def version_to_float(version):
    if pd.isna(version):
        return np.nan
    parts = version.strip().split('.')
    parts = [int(p) if p.isdigit() else 0 for p in parts]
    while len(parts) < 4:
        parts.append(0)
    return float(f"{parts[0]}.{parts[1]:02d}{parts[2]:02d}{parts[3]:02d}")

df['Current Version Num'] = df['Current Version'].apply(version_to_float)
df['Android Version Clean'] = df['Android Version'].str.replace('and up', '', regex=False).str.strip()
df['Android Version Num'] = df['Android Version Clean'].apply(version_to_float)



In [233]:
df.loc[df['Current Version'] == 'Varies with device', 'Current Version Num'] = -1
df['Android Version Num'].replace(-1, np.nan, inplace=True)


mean_val = df['Android Version Num'].mean()
df['Android Version Num'].fillna(mean_val, inplace=True)

df['Current Version Num'].replace(-1, np.nan, inplace=True)


mean_val = df['Current Version Num'].mean()
df['Current Version Num'].fillna(mean_val, inplace=True)


In [234]:
df[['Current Version', 'Current Version Num']]

,Current Version,Current Version Num
0,1.9.2,1.090200
1,2.1.0,2.010000
2,1.1,1.010000
3,1.9,1.090000
4,2.0.10,2.001000
...,...,...
1868,2.10.3,2.100300
1869,1.0,1.000000
1870,Varies with device,12775.942475
1871,6.3.2,6.030200


In [235]:
df['Current Version Num'].fillna(df['Current Version Num'].median(), inplace=True)
df['Android Version Num'].fillna(df['Android Version Num'].median(), inplace=True)


In [236]:
df[['Current Version', 'Current Version Num']]

,Current Version,Current Version Num
0,1.9.2,1.090200
1,2.1.0,2.010000
2,1.1,1.010000
3,1.9,1.090000
4,2.0.10,2.001000
...,...,...
1868,2.10.3,2.100300
1869,1.0,1.000000
1870,Varies with device,12775.942475
1871,6.3.2,6.030200


In [237]:
df[['Android Version', 'Android Version Num']]

,Android Version,Android Version Num
0,4.0.3 and up,4.0003
1,4.1 and up,4.0100
2,4.0.3 and up,4.0003
3,2.3 and up,2.0300
4,4.0.3 and up,4.0003
...,...,...
1868,2.3.3 and up,2.0303
1869,2.0 and up,2.0000
1870,Varies with device,0.0000
1871,Varies with device,0.0000


In [238]:
df.isnull().sum()

App Name                 0
Category                 0
Reviews                  0
Size                     0
Installs                 0
Type                     0
Price                    0
Content Rating           0
Genre                    0
Current Version          1
Android Version          0
Days_Since_Update        0
Is_Varies_Android        0
Is_Varies_Current        0
Current Version Num      0
Android Version Clean    0
Android Version Num      0
dtype: int64

In [239]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873 entries, 0 to 1872
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   App Name               1873 non-null   object 
 1   Category               1873 non-null   object 
 2   Reviews                1873 non-null   int64  
 3   Size                   1873 non-null   object 
 4   Installs               1873 non-null   int64  
 5   Type                   1873 non-null   object 
 6   Price                  1873 non-null   float64
 7   Content Rating         1873 non-null   object 
 8   Genre                  1873 non-null   object 
 9   Current Version        1872 non-null   object 
 10  Android Version        1873 non-null   object 
 11  Days_Since_Update      1873 non-null   int64  
 12  Is_Varies_Android      1873 non-null   bool   
 13  Is_Varies_Current      1873 non-null   bool   
 14  Current Version Num    1873 non-null   float64
 15  Andr

In [240]:
df.head()

,App Name,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genre,Current Version,Android Version,Days_Since_Update,Is_Varies_Android,Is_Varies_Current,Current Version Num,Android Version Clean,Android Version Num
0,Girls hairstyles 2018,BEAUTY,62,3.1M,10000,Free,0.0,Everyone,Beauty,1.9.2,4.0.3 and up,71,False,False,1.0902,4.0.3,4.0003
1,Dairy Queen,FOOD_AND_DRINK,742,43M,100000,Free,0.0,Everyone,Food & Drink,2.1.0,4.1 and up,13,False,False,2.0100,4.1,4.0100
2,Remote Control For All AC - Universal Remote,PRODUCTIVITY,166,6.1M,10000,Free,0.0,Everyone,Productivity,1.1,4.0.3 and up,10,False,False,1.0100,4.0.3,4.0003
3,Ultimate Chest Tracker,PRODUCTIVITY,40328,23M,1000000,Free,0.0,Everyone,Productivity,1.9,2.3 and up,659,False,False,1.0900,2.3,2.0300
4,Mobilight-BM,FINANCE,6,6.2M,500,Free,0.0,Everyone,Finance,2.0.10,4.0.3 and up,4,False,False,2.0010,4.0.3,4.0003


Type

In [241]:
df['Type'].unique()

array(['Free', 'Paid'], dtype=object)

In [242]:
# Check if all Free apps have Price = 0
free_check = df[df['Type'] == 'Free']['Price'].eq(0.0).all()

# Check if all Paid apps have Price > 0
paid_check = df[df['Type'] == 'Paid']['Price'].gt(0.0).all()

print("Free apps have Price 0:", free_check)
print("Paid apps have Price > 0:", paid_check)


Free apps have Price 0: True
Paid apps have Price > 0: True


In [243]:
df.drop(columns=['Type'], inplace=True)


Category

In [244]:
df['genre_list'] = df['Genre'].apply(lambda x: x.split(';'))
df['genre_list']

0              [Beauty]
1        [Food & Drink]
2        [Productivity]
3        [Productivity]
4             [Finance]
             ...       
1868          [Finance]
1869            [Tools]
1870           [Sports]
1871    [Communication]
1872           [Action]
Name: genre_list, Length: 1873, dtype: object

In [245]:

mlb = joblib.load('mlb.pkl')
# Transform the new genres using the already fitted mlb
new_genre_encoded = mlb.transform(df['genre_list'])

# Create a DataFrame from the encoded genre data
new_genre_encoded_df = pd.DataFrame(new_genre_encoded, columns=mlb.classes_, index=df.index)

# Concatenate with the new_df (drop genre_list if needed)
df = pd.concat([df, new_genre_encoded_df], axis=1)
df.drop(columns='genre_list', inplace=True)  # optional

In [246]:
df

,App Name,Category,Reviews,Size,Installs,Price,Content Rating,Genre,Current Version,Android Version,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,Girls hairstyles 2018,BEAUTY,62,3.1M,10000,0.0,Everyone,Beauty,1.9.2,4.0.3 and up,...,0,0,0,0,0,0,0,0,0,0
1,Dairy Queen,FOOD_AND_DRINK,742,43M,100000,0.0,Everyone,Food & Drink,2.1.0,4.1 and up,...,0,0,0,0,0,0,0,0,0,0
2,Remote Control For All AC - Universal Remote,PRODUCTIVITY,166,6.1M,10000,0.0,Everyone,Productivity,1.1,4.0.3 and up,...,0,0,0,0,0,0,0,0,0,0
3,Ultimate Chest Tracker,PRODUCTIVITY,40328,23M,1000000,0.0,Everyone,Productivity,1.9,2.3 and up,...,0,0,0,0,0,0,0,0,0,0
4,Mobilight-BM,FINANCE,6,6.2M,500,0.0,Everyone,Finance,2.0.10,4.0.3 and up,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,FREE Stock Market Trading Tips,FINANCE,714,3.6M,50000,0.0,Everyone,Finance,2.10.3,2.3.3 and up,...,0,0,0,0,0,0,0,0,0,0
1869,Next Portuguese(BR) Langpack,TOOLS,1320,778k,100000,0.0,Everyone,Tools,1.0,2.0 and up,...,0,0,0,0,1,0,0,0,0,0
1870,850 Sports News Digest,SPORTS,539,Varies with device,10000,0.0,Everyone,Sports,Varies with device,Varies with device,...,0,0,1,0,0,0,0,0,0,0
1871,Lite for Facebook Messenger,COMMUNICATION,76498,4.3M,1000000,0.0,Teen,Communication,6.3.2,Varies with device,...,0,0,0,0,0,0,0,0,0,0


In [247]:
df['Category'].unique()

array(['BEAUTY', 'FOOD_AND_DRINK', 'PRODUCTIVITY', 'FINANCE', 'FAMILY',
       'LIFESTYLE', 'VIDEO_PLAYERS', 'SHOPPING', 'GAME', 'TOOLS',
       'COMMUNICATION', 'EDUCATION', 'BUSINESS', 'COMICS', 'DATING',
       'BOOKS_AND_REFERENCE', 'NEWS_AND_MAGAZINES', 'PARENTING',
       'PERSONALIZATION', 'MEDICAL', 'HEALTH_AND_FITNESS',
       'TRAVEL_AND_LOCAL', 'PHOTOGRAPHY', 'SPORTS', 'LIBRARIES_AND_DEMO',
       'HOUSE_AND_HOME', 'SOCIAL', 'WEATHER', 'EVENTS',
       'AUTO_AND_VEHICLES', 'ENTERTAINMENT', 'ART_AND_DESIGN',
       'MAPS_AND_NAVIGATION'], dtype=object)

In [248]:
categories = ['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY',
              'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION',
              'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE',
              'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME',
              'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL',
              'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL',
              'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER',
              'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION']


category_map = {cat: i for i, cat in enumerate(categories)}


df['Category_Encoded'] = df['Category'].map(category_map)

In [249]:
df['Cat_Categorical'] = df['Category']

df = pd.get_dummies(df, columns=['Category'], prefix='One_Hot_Cat')

In [250]:
df.head(100)

,App Name,Reviews,Size,Installs,Price,Content Rating,Genre,Current Version,Android Version,Days_Since_Update,...,One_Hot_Cat_PERSONALIZATION,One_Hot_Cat_PHOTOGRAPHY,One_Hot_Cat_PRODUCTIVITY,One_Hot_Cat_SHOPPING,One_Hot_Cat_SOCIAL,One_Hot_Cat_SPORTS,One_Hot_Cat_TOOLS,One_Hot_Cat_TRAVEL_AND_LOCAL,One_Hot_Cat_VIDEO_PLAYERS,One_Hot_Cat_WEATHER
0,Girls hairstyles 2018,62,3.1M,10000,0.0,Everyone,Beauty,1.9.2,4.0.3 and up,71,...,False,False,False,False,False,False,False,False,False,False
1,Dairy Queen,742,43M,100000,0.0,Everyone,Food & Drink,2.1.0,4.1 and up,13,...,False,False,False,False,False,False,False,False,False,False
2,Remote Control For All AC - Universal Remote,166,6.1M,10000,0.0,Everyone,Productivity,1.1,4.0.3 and up,10,...,False,False,True,False,False,False,False,False,False,False
3,Ultimate Chest Tracker,40328,23M,1000000,0.0,Everyone,Productivity,1.9,2.3 and up,659,...,False,False,True,False,False,False,False,False,False,False
4,Mobilight-BM,6,6.2M,500,0.0,Everyone,Finance,2.0.10,4.0.3 and up,4,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Google Analytics,78662,22M,1000000,0.0,Everyone,Business,3.7.5,4.4 and up,175,...,False,False,False,False,False,False,False,False,False,False
96,Forgotten Hill: Surgery,2431,24M,100000,0.0,Teen,Adventure,1.4,3.0 and up,277,...,False,False,False,False,False,False,False,False,False,False
97,Weather BZ,8433,5.6M,100000,0.0,Everyone,Weather,5.0.1 build 1,4.0 and up,62,...,False,False,False,False,False,False,False,False,False,True
98,Car Parking Crane N Drifting,106,71M,50000,0.0,Everyone,Simulation,1.8,4.0.3 and up,162,...,False,False,False,False,False,False,False,False,False,False


In [251]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873 entries, 0 to 1872
Columns: 103 entries, App Name to One_Hot_Cat_WEATHER
dtypes: bool(35), float64(3), int64(57), object(8)
memory usage: 1.0+ MB


In [252]:
for col in df.columns:
    if col.startswith('One_Hot_Cat') or col.startswith('Is_Varies'):
        df[col] = df[col].astype(int)

In [253]:
df.drop(columns=['Current Version','Android Version','Android Version Clean'], inplace=True)

In [254]:
df

,App Name,Reviews,Size,Installs,Price,Content Rating,Genre,Days_Since_Update,Is_Varies_Android,Is_Varies_Current,...,One_Hot_Cat_PERSONALIZATION,One_Hot_Cat_PHOTOGRAPHY,One_Hot_Cat_PRODUCTIVITY,One_Hot_Cat_SHOPPING,One_Hot_Cat_SOCIAL,One_Hot_Cat_SPORTS,One_Hot_Cat_TOOLS,One_Hot_Cat_TRAVEL_AND_LOCAL,One_Hot_Cat_VIDEO_PLAYERS,One_Hot_Cat_WEATHER
0,Girls hairstyles 2018,62,3.1M,10000,0.0,Everyone,Beauty,71,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dairy Queen,742,43M,100000,0.0,Everyone,Food & Drink,13,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Remote Control For All AC - Universal Remote,166,6.1M,10000,0.0,Everyone,Productivity,10,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Ultimate Chest Tracker,40328,23M,1000000,0.0,Everyone,Productivity,659,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Mobilight-BM,6,6.2M,500,0.0,Everyone,Finance,4,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,FREE Stock Market Trading Tips,714,3.6M,50000,0.0,Everyone,Finance,426,0,0,...,0,0,0,0,0,0,0,0,0,0
1869,Next Portuguese(BR) Langpack,1320,778k,100000,0.0,Everyone,Tools,1789,0,0,...,0,0,0,0,0,0,1,0,0,0
1870,850 Sports News Digest,539,Varies with device,10000,0.0,Everyone,Sports,714,1,1,...,0,0,0,0,0,1,0,0,0,0
1871,Lite for Facebook Messenger,76498,4.3M,1000000,0.0,Teen,Communication,48,1,0,...,0,0,0,0,0,0,0,0,0,0


In [255]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873 entries, 0 to 1872
Data columns (total 100 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   App Name                         1873 non-null   object 
 1   Reviews                          1873 non-null   int64  
 2   Size                             1873 non-null   object 
 3   Installs                         1873 non-null   int64  
 4   Price                            1873 non-null   float64
 5   Content Rating                   1873 non-null   object 
 6   Genre                            1873 non-null   object 
 7   Days_Since_Update                1873 non-null   int64  
 8   Is_Varies_Android                1873 non-null   int64  
 9   Is_Varies_Current                1873 non-null   int64  
 10  Current Version Num              1873 non-null   float64
 11  Android Version Num              1873 non-null   float64
 12  Action             

In [256]:
df['Reviews'].info()
df['Reviews'] = df['Reviews'].astype(int)


<class 'pandas.core.series.Series'>
RangeIndex: 1873 entries, 0 to 1872
Series name: Reviews
Non-Null Count  Dtype
--------------  -----
1873 non-null   int64
dtypes: int64(1)
memory usage: 14.8 KB


In [257]:
df['Reviews_log'] = np.log1p(df['Reviews'])
df['Installs_log'] = np.log1p(df['Installs'])
df['Price_log'] = np.log1p(df['Price'])
df['Days_Since_Update_log'] = np.log1p(df['Days_Since_Update'])


In [258]:
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', '')) * 1_000  # MB to KB
    elif 'k' in size:
        return float(size.replace('k', ''))


df['Size'] = df['Size'].apply(convert_size)
df['Size_log'] = np.log(df['Size'])

In [259]:
df['Is_Free'] = (df['Price'] == 0).astype(int)

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1873 entries, 0 to 1872
Columns: 106 entries, App Name to Is_Free
dtypes: float64(9), int64(93), object(4)
memory usage: 1.5+ MB


In [261]:
df['Content Rating'].unique()

array(['Everyone', 'Teen', 'Mature 17+', 'Everyone 10+'], dtype=object)

In [262]:
rating_order = {
    'Everyone': 0,
    'Everyone 10+': 1,
    'Teen': 2,
    'Mature 17+': 3,
    'Adults only 18+': 4,
    'Unrated': -1  # or 5, depending on how you want to treat "Unrated"
}

In [263]:
df['content_rating_ordinal'] = df['Content Rating'].map(rating_order)

In [264]:
df['content_rating_Categorical'] = df['Content Rating']

In [265]:
df = pd.get_dummies(df, columns=['Content Rating'], prefix='rating')

In [266]:
df_temp = df.copy()
df_temp.drop(columns=['App Name', 'content_rating_Categorical', 'Cat_Categorical','Genre'], inplace=True)

In [267]:
columns_to_use = ['Reviews', 'Size', 'Installs', 'Price']

In [268]:


# Normalize the columns using MinMaxScaler
#scaler = joblib.load('scaler.pkl')
#df[columns_to_use] = scaler.transform(df[columns_to_use])


In [269]:
for column in df.columns:
    null_count = df[column].isnull().sum()
    if null_count > 0:
        print(f"{column}: {null_count} null values")


Size: 336 null values
Size_log: 336 null values


In [284]:
df['Size'] = df['Size'].fillna(22768)


In [270]:

target               = ['Y']

numeric_raw          = ['Reviews', 'Size', 'Installs', 'Price',
                        'Days_Since_Update', 'Current Version Num',
                        'Android Version Num', 'Is_Varies_Android', 'Is_Free']

numeric_log          = ['Reviews_log', 'Price_log',
                        'Days_Since_Update_log', 'Size_log']

genre_onehot         = ['Action', 'Action & Adventure', 'Adventure', 'Arcade',
                        'Art & Design', 'Auto & Vehicles', 'Beauty', 'Board',
                        'Books & Reference', 'Brain Games', 'Business', 'Card',
                        'Casino', 'Casual', 'Comics', 'Communication',
                        'Creativity', 'Dating', 'Education', 'Educational',
                        'Entertainment', 'Events', 'Finance', 'Food & Drink',
                        'Health & Fitness', 'House & Home',
                        'Libraries & Demo', 'Lifestyle', 'Maps & Navigation',
                        'Medical', 'Music', 'Music & Audio', 'Music & Video',
                        'News & Magazines', 'Parenting', 'Personalization',
                        'Photography', 'Pretend Play', 'Productivity', 'Puzzle',
                        'Racing', 'Role Playing', 'Shopping', 'Simulation',
                        'Social', 'Sports', 'Strategy', 'Tools',
                        'Travel & Local', 'Trivia',
                        'Video Players & Editors', 'Weather', 'Word']

category_ordinal     = ['Category_Encoded']
category_label       = ['Cat_Categorical']
category_onehot      = ['One_Hot_Cat_EDUCATION', 'One_Hot_Cat_ENTERTAINMENT',
                        'One_Hot_Cat_FAMILY', 'One_Hot_Cat_GAME']

content_rating_ord   = ['content_rating_ordinal']
content_rating_lbl   = ['content_rating_Categorical']
content_rating_one   = [ 'rating_Everyone',
                        'rating_Everyone 10+', 'rating_Mature 17+',
                        'rating_Teen']




In [271]:
features_num_raw     = numeric_raw                      # 9 vars
features_num_log     = numeric_log                      # 4 vars
features_num_both    = numeric_raw + numeric_log        # 13 vars


In [272]:
features_ord_minimal = numeric_raw + category_ordinal + content_rating_ord
features_ord_full    = numeric_raw + numeric_log + category_ordinal + content_rating_ord
features_ord         = numeric_log + category_ordinal + content_rating_ord
features_cat1h       = numeric_raw + category_onehot
features_genre1h     = numeric_raw + genre_onehot
features_full1h      = numeric_raw + numeric_log + genre_onehot + category_onehot + content_rating_one
features_balanced    = numeric_raw + numeric_log + genre_onehot + category_ordinal + content_rating_ord
features_balanced2    = numeric_raw + category_ordinal + content_rating_ord + genre_onehot


In [282]:
rf = joblib.load('balanced2RF_700_depthNone_frac.pkl')
tokenizer = joblib.load('tokenizer.pkl')
# Load DL model
from tensorflow.keras.models import load_model
model = load_model('dl_model.h5')

# Load meta model
meta_model = joblib.load('meta_model.pkl')


In [285]:


x = df[features_balanced2]

In [286]:
from keras.src.utils import pad_sequences

# RF prediction
rf_pred = rf.predict(x)
new_data_name = df['App Name']

# DL prediction
new_name_seq = tokenizer.texts_to_sequences(new_data_name)
new_name_pad = pad_sequences(new_name_seq, padding='post', maxlen=10)
dl_pred = model.predict(new_name_pad).flatten()

# Stack and predict
stacked_input = np.vstack([rf_pred, dl_pred]).T
final_prediction = meta_model.predict(stacked_input)


59/59 [==============================] - 0s 306us/step


In [288]:

row_ids = x.index  # or any saved 'row_id' column

submission_df = pd.DataFrame({
    'row_id': row_ids,
    'Y': final_prediction
})

submission_df.to_csv('predictions.csv', index=False)

In [289]:

# Load your two CSV files
df_y = pd.read_csv('predictions.csv')      # This has the Y values only
df_ids = pd.read_csv('data/SampleSubmission.csv')    # This has the row_id values only

# Combine them by index (ignore their internal row_id/Y columns)
combined = pd.DataFrame({
    'row_id': df_ids['row_id'],
    'Y': df_y['Y']
})

# Save result if needed
combined.to_csv('data/combined_output.csv', index=False)

print(combined.head())


   row_id         Y
0    2933  4.241581
1     734  4.128304
2    1139  4.024010
3    9394  4.170502
4    7393  4.301143
